In [1]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from collections import Counter
import os
import random
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [3]:
path = '/home/aremirata/thesis/DIARETB30/diaretdb0_v_1_1/resources/images/'
images_list = np.sort(os.listdir(path+'diaretdb0_fundus_images/'))
labels_files = np.sort(os.listdir(path+'diaretdb0_groundtruths/'))

In [4]:
image_arrays = []
image_name = []
for i in images_list:
    img = Image.open(path+'diaretdb0_fundus_images/'+i)
    img_rs = img.resize((256,256), Image.ANTIALIAS)
    a = np.asarray(img_rs)
    image_arrays.append(a)
    image_name.append(i)

In [5]:
image_arrays = np.asarray(image_arrays)

In [6]:
target_labels = []
for i in labels_files:
    text_file = open(path+'diaretdb0_groundtruths/'+i, "r")
    target_labels.append(np.asarray(text_file.read().lower().split()))

In [7]:
target_labels = np.asarray(target_labels)

In [8]:
np.unique(target_labels)

array(['hardexudates', 'hemorrhages', 'n/a', 'neovascularisation',
       'redsmalldots', 'softexudates'], dtype='<U18')

In [9]:
print(target_labels.shape)

(130, 5)


In [10]:
print(image_arrays.shape)

(130, 256, 256, 3)


In [11]:
target_labels_encoded = []
for i in target_labels:
    l = np.asarray([0 if j == 'n/a' else 1 for j in i])
    target_labels_encoded.append(l)

In [12]:
target_labels_encoded = np.asarray(target_labels_encoded)

In [13]:
target_labels_encoded.shape

(130, 5)

In [14]:
target_labels_modified = np.array([i/float(j) if j!=0 else i for i,j in zip(target_labels_encoded,np.sum(target_labels_encoded,axis=1))])

In [15]:
modified_classes = np.asarray([[1,0] if i==0 else [0,1] for i in np.sum(target_labels_encoded,axis=1)])

In [16]:
classes_unique = []
for i in range(target_labels.shape[1]):
    classes_unique = classes_unique + list(np.unique(target_labels[:,i])[np.unique(target_labels[:,i]) != 'n/a'])

In [17]:
classes_unique

['redsmalldots',
 'hemorrhages',
 'hardexudates',
 'softexudates',
 'neovascularisation']

In [18]:
modified_class_labels = ['normal','infected']

In [19]:
normal_retina_ids = np.argwhere(modified_classes[:,0]==1).reshape([-1])
affected_retina_ids = np.argwhere(modified_classes[:,0]==0).reshape([-1])

In [20]:
pathn = '/home/aremirata/thesis/DIARETB30/diaretdb0_v_1_1/resources/'
train_batch_files = os.listdir(pathn+'traindatasets/')
test_batch_files = os.listdir(pathn+'testdatasets/')

In [21]:
label_dict = {'normal':normal_retina_ids, 'affected': affected_retina_ids}

In [22]:
label_dict

{'affected': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
        106, 107, 108, 109]),
 'normal': array([ 81,  94, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
        121, 122, 123, 124, 125, 126, 127, 128, 129])}

In [23]:
label_df = pd.DataFrame([label_dict]).transpose().reset_index().rename(columns={0:'indices','index':'label'})

In [24]:
label_df

,label,indices
0,affected,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,normal,"[81, 94, 110, 111, 112, 113, 114, 115, 116, 11..."


In [25]:
label_data = label_df.apply(lambda x: pd.Series(x['indices']), axis=1).stack().reset_index(level=1, drop=True)

In [26]:
label_data.name = 'index'

In [27]:
label_dataframe = label_df.drop('indices', axis=1).join(label_data)

In [28]:
targetmap = {'normal':0, 'affected':1}

In [29]:
label_dataframe['image_name'] = label_dataframe['index'].apply(lambda x: 'image'+str(int(x) + 1).zfill(3)+'.png')
label_dataframe['target'] = label_dataframe['label'].apply(lambda x: targetmap[x])
del label_dataframe['index']

In [30]:
label_dataframe.head()

,label,image_name,target
0,affected,image001.png,1
0,affected,image002.png,1
0,affected,image003.png,1
0,affected,image004.png,1
0,affected,image005.png,1


In [31]:
index = range(0, len(image_arrays))
df_image = pd.DataFrame(index=index, columns=['image_name','image'])
for i in range(len(image_arrays)):
    df_image['image_name'][i] = image_name[i]
    df_image['image'][i] = image_arrays[i]

In [32]:
df_image.tail()

,image_name,image
125,image126.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
126,image127.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
127,image128.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
128,image129.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
129,image130.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."


In [33]:
dataset_df = pd.merge(df_image,label_dataframe,on='image_name',how='inner')

In [34]:
dataset_df.head()

,image_name,image,label,target
0,image001.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",affected,1
1,image002.png,"[[[1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], ...",affected,1
2,image003.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",affected,1
3,image004.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",affected,1
4,image005.png,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",affected,1


# Deep Learning For DR Classification

In [35]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class custom_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
    def on_train_begin(self, logs={}):
        return
    def on_train_end(self, logs={}):
        return
    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        logs["roc-auc"] = roc
        logs["roc-auc_val"] = roc_val
        return
    def on_batch_begin(self, batch, logs={}):
        return
    def on_batch_end(self, batch, logs={}):
        return

In [36]:
x_training, x_test, y_training, y_test = train_test_split(np.stack(dataset_df['image'].as_matrix(), axis=0), 
                                                    np.stack(dataset_df['target'].as_matrix(), axis=0), 
                                                    test_size=0.33, random_state=42)

In [37]:
x_train, x_val, y_train, y_val = train_test_split(x_training, y_training, 
                                                    test_size=0.33, random_state=42)

In [38]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)
y_val = keras.utils.to_categorical(y_val, 2)

In [39]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((58, 256, 256, 3), (43, 256, 256, 3), (58, 2), (43, 2))

In [40]:
batch_size = 32
num_classes = 2
epochs = 100
data_augmentation = True
num_predictions = 10

In [41]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [42]:
# Let's train the model using rmsprop
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    cnn_callback = model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=10,
                        epochs=epochs,
                        callbacks = [custom_callback(
                                training_data=(x_train, y_train),
                                validation_data=(x_val, y_val))],
                        validation_data=(x_val, y_val),
                        use_multiprocessing=True,
                        workers=4)

score = model.evaluate(x_val, y_val, verbose=0)
    
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

pred_test = model.predict(x_test)
roc_score_test = roc_auc_score(y_test, pred_test)


Using real-time data augmentation.


/home/aremirata/.conda/envs/t2d_pred/lib/python3.5/site-packages/keras/engine/training.py:1987: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/100
roc-auc: 0.4564 - roc-auc_val: 0.48                                                                                                    
10/10 [==============================] - 8s - loss: 1.1125 - acc: 0.7828 - val_loss: 2.2111 - val_acc: 0.8621
Epoch 2/100
roc-auc: 0.4786 - roc-auc_val: 0.56                                                                                                    

In [43]:
roc_score_test

0.7986842105263159